### Pete Champlin
### Big Data 230 A
#### Week 6 Assignment
5/25/2020

## Data Source
We will start by reading in the [Kaggle Boston Housing dataset](https://www.kaggle.com/c/boston-housing/data).

![](https://storage.googleapis.com/kaggle-competitions/kaggle/5315/logos/front_page.png)

**Housing Values in Suburbs of Boston**
The medv variable is the target variable.

**Data description**
The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

| columns | description |
| :------- | :----------- |
| crim |  per capita crime rate by town. |
| zn | proportion of residential land zoned for lots over 25,000 sq.ft. |
| indus | proportion of non-retail business acres per town. |
| chas |  Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). |
| nox |  nitrogen oxides concentration (parts per 10 million). |
| rm | average number of rooms per dwelling. |
| age |  proportion of owner-occupied units built prior to 1940. |
| dis |  weighted mean of distances to five Boston employment centres. |
| rad |  index of accessibility to radial highways. |
| tax | full-value property-tax rate per \$10,000. |
| ptratio | pupil-teacher ratio by town. |
| black | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town. |
| lstat | lower status of the population (percent). |
| medv | median value of owner-occupied homes in \$1000s. |

Sources:
* Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.
* Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.

Resources:
* A great more indepth blog post is Susan Li's [Building A Linear Regression with PySpark and MLlib](https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a)

### Download the Boston Housing dataset

#### Important: Change the folder you are writing the file

In [4]:
%sh
rm /dbfs/tmp/pbchamp/samples/boston/*
mkdir -p /dbfs/tmp/pbchamp/samples/boston/
wget -O /dbfs/tmp/pbchamp/samples/boston/boston-housing.csv https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/boston-housing.csv
ls -al /dbfs/tmp/pbchamp/samples/boston/

--2020-05-26 03:54:51-- https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/boston-housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.40.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35818 (35K) [text/plain]
Saving to: ‘/dbfs/tmp/pbchamp/samples/boston/boston-housing.csv’

 0K .......... .......... .......... .... 100% 3.90M=0.009s

2020-05-26 03:54:52 (3.90 MB/s) - ‘/dbfs/tmp/pbchamp/samples/boston/boston-housing.csv’ saved [35818/35818]

total 43
drwxrwxrwx 2 root root 4096 May 26 03:54 .
drwxrwxrwx 2 root root 4096 May 14 02:26 ..
-rwxrwxrwx 1 root root 35818 May 26 03:54 boston-housing.csv

## Load Data Using Pandas Syntax via Koalas
We can load our data using the Pandas syntax `.read_csv()` to read the CSV file and use the Pandas syntax `.head()` to review the top 5 rows of our Spark DataFrame.

In [6]:
import pandas as pd

dbfs_dir = '/dbfs/tmp/pbchamp/samples/boston'
dbfs_file = 'boston-housing.csv'
dbfs_path = dbfs_dir + '/' + dbfs_file

pdf = pd.read_csv(dbfs_path)

# New column names
column_names = ['ID', 'crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status', 'median_value']

# Rename columns
pdf.columns = column_names

#Keep rows where median value is not null
pdf = pdf[pdf['median_value'].notnull()]

pdf.head(5)

,ID,crime,zone,industry,bounds_river,nox,rooms,age,distance,radial_highway,tax,pupil_teacher,black_proportion,lower_status,median_value
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [7]:
#Not needed for this exercise, just a reminder that it's possible

import databricks.koalas as ks

# Convert pandas dataframe to Koalas DataFrame
kdf = ks.DataFrame(pdf)

kdf.head(5)

,ID,crime,zone,industry,bounds_river,nox,rooms,age,distance,radial_highway,tax,pupil_teacher,black_proportion,lower_status,median_value
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9


In [8]:
# Get the features most-highly correlated (over the correlation_threshold) 
#  with the target to predict, in descending order (by absolute value)

correlation_threshold = 0.40 #Change as desired

# Calculate using Pandas `corr`
pdf_corr = kdf.drop('ID').toPandas().corr()

# Add Index 
pdf_corr['feature'] = pdf_corr.index
corr = pdf_corr.loc[:, ['feature', 'median_value']]
corr = corr.rename({'median_value':'correlation'}, axis='columns')
corr = corr[abs(corr['correlation']) > correlation_threshold].drop('median_value')
corr['correlation'] = abs(corr['correlation'])
features = corr.sort_values(by='correlation', ascending=False)
display(features)

feature,correlation
lower_status,0.738600034878634
rooms,0.6895980892872157
pupil_teacher,0.48137595552491874
industry,0.47393197065920345
tax,0.4480776944007039
nox,0.41305415199207746
crime,0.4074543235732594


In [9]:
#Execute model runs, using various model types and features

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
from yellowbrick.regressor import PredictionError, ResidualsPlot

# Re-generate Pandas DataFrame from updated Koala DataFrame
pdf = kdf.toPandas()

random_seed = 34729
test_size = 0.25
target = 'median_value'
log_prediction_error = True
log_residuals_plot = True

lr_models = ['Ridge', 'Lasso']
features_run = []

# Iterate through model types
for model in lr_models:
  
  # Run iterations, adding a new feature for each run
  for feature in features['feature']:
    features_run.append(feature) # Add the next highest-correlated feature

    run_name = model + str(len(features_run))
    print('run name: ', run_name)
    print('features: ', features_run)
    
    # Create train/test datasets based on this run's features
    X_train, X_test, y_train, y_test = (
      train_test_split( 
        pdf[features_run].values, 
        pdf[target].values, 
        test_size=test_size, 
        random_state=random_seed)
    )
    
    # Create linear regression object
    if model == 'Ridge':
      lr = Ridge()
    elif model == 'Lasso':
      lr = Lasso()

    # Start this MLFlow run's context
    with mlflow.start_run(run_name=run_name) as run:
      #print('run_id: ' + str(run.info.run_id))
      mlflow.set_tag('model', model)
      mlflow.set_tag('features', features_run)
    
      # Fit and predict
      lr.fit(X_train, y_train)
      y_pred = lr.predict(X_test)

      #print(y_test[:10])
      #print(y_pred[:10])

      # Get model evaluation metrics
      rmse = np.sqrt(mean_squared_error(y_test, y_pred))
      r2 = r2_score(y_test, y_pred)

      # Log to MLFlow
      mlflow.log_metric("rmse", rmse)
      mlflow.log_metric("r2", r2)
      mlflow.sklearn.log_model(lr, "model")
      
      # Create and log model evaluation plots
      if log_prediction_error == True:
        image_name_pe = run_name + '_PredictionError.png'
        visualizer_pe = PredictionError(lr, size=(1000,800))
        visualizer_pe.fit(X_train, y_train)  # Fit the training data to the visualizer
        visualizer_pe.score(X_test, y_test)  # Evaluate the model on the test data
        visualizer_pe.show(outpath=image_name_pe, clear_figure=True)
        mlflow.log_artifact(image_name_pe)
        #visualizer_pe.poof()                 # Finalize and render the figure
      
      if log_residuals_plot == True:
        image_name_re = run_name + '_Residuals.png'          
        visualizer_re = ResidualsPlot(lr, size=(1000, 800))
        visualizer_re.fit(X_train, y_train)  # Fit the training data to the model
        visualizer_re.score(X_test, y_test)  # Evaluate the model on the test data
        visualizer_re.show(outpath=image_name_re, clear_figure=True)
        mlflow.log_artifact(image_name_re)
        #visualizer_re.poof()                 # Draw/show/poof the data      
      
  features_run.clear()


run name: Ridge1
features: ['lower_status']
run name: Ridge2
features: ['lower_status', 'rooms']
run name: Ridge3
features: ['lower_status', 'rooms', 'pupil_teacher']
run name: Ridge4
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry']
run name: Ridge5
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax']
run name: Ridge6
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax', 'nox']
run name: Ridge7
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax', 'nox', 'crime']
run name: Lasso1
features: ['lower_status']
run name: Lasso2
features: ['lower_status', 'rooms']
run name: Lasso3
features: ['lower_status', 'rooms', 'pupil_teacher']
run name: Lasso4
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry']
run name: Lasso5
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax']
run name: Lasso6
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax', 'nox']
run name: Lasso7
features: ['lower_status', 'rooms', 'pupil_teacher', 'industry', 'tax', 'nox', 'crime']

In [10]:
#Get the best-performing model by R2

from mlflow.tracking.client import MlflowClient
from mlflow.entities import ViewType

run = MlflowClient().search_runs(
  experiment_ids="2295836680664125", 
  filter_string="",
  run_view_type=ViewType.ACTIVE_ONLY,
  max_results=2,
  order_by=["metrics.r2 DESC"]
)[0]
run


Out[138]: <Run: data=<RunData: metrics={'r2': 0.5635051237858122, 'rmse': 6.359999930174006}, params={}, tags={'features': "['lower_status', 'rooms', 'pupil_teacher']",
 'mlflow.databricks.notebookID': '2295836680664125',
 'mlflow.databricks.notebookPath': '/Users/pbchamp@uw.edu/230/BostonHousing_MLFlow_Week6',
 'mlflow.databricks.notebookRevisionID': '1590467374153',
 'mlflow.databricks.webappURL': 'https://westus.azuredatabricks.net',
 'mlflow.log-model.history': '[{"run_id":"4e7c7c915e8d454596be644f4011c28f","artifact_path":"model","utc_time_created":"2020-05-26 '
 '04:29:32.892969","flavors":{"python_function":{"loader_module":"mlflow.sklearn","python_version":"3.7.3","data":"model.pkl","env":"conda.yaml"},"sklearn":{"pickled_model":"model.pkl","sklearn_version":"0.20.3","serialization_format":"cloudpickle"}}}]',
 'mlflow.runName': 'Lasso3',
 'mlflow.source.name': '/Users/pbchamp@uw.edu/230/BostonHousing_MLFlow_Week6',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'pbchamp@uw.edu',
 'model': 'Lasso'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow/2295836680664125/4e7c7c915e8d454596be644f4011c28f/artifacts', end_time=1590467374053, experiment_id='2295836680664125', lifecycle_stage='active', run_id='4e7c7c915e8d454596be644f4011c28f', run_uuid='4e7c7c915e8d454596be644f4011c28f', start_time=1590467372437, status='FINISHED', user_id=''>>

#### Root Mean Square Error (RMSE)

Root Mean Square Error (RMSE) is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.

In [12]:
#Show RMSE calculation, just as a reinforcement exercise

import numpy as np

rmse2 = np.sqrt(np.mean((y_test - y_pred) ** 2))
print(rmse)
print(rmse2)

6.400631498743966
6.400631498743966

#### Coefficient of Determination (R Squared)

The coefficient of determination, R2, is used to analyze how differences in one variable can be explained by a difference in a second variable. For example, when a person gets pregnant has a direct relation to when they give birth.

More specifically, R-squared gives you the percentage variation in y explained by x-variables. The range is 0 to 1 (i.e. 0% to 100% of the variation in y can be explained by the x-variables.

The coefficient of determination, R2, is similar to the correlation coefficient, R. The correlation coefficient formula will tell you how strong of a linear relationship there is between two variables. R Squared is the square of the correlation coefficient, r (hence the term r squared).